This is still very much rough draft. The dependent variable is the difference in segregation when measured in euclidean dist versus network dist. There are a couple strategies we could take:

how to measure difference:

- raw difference
- percent difference 

how to model:

- take the whole dataset and regress seg difference on network measures
- take subset of significant results and regress
- take whole dataset and use logistic regression to see which metrics result in a significant difference

not necessarily mututally exclusive...

to finish the modeling, we also need some additional variables, 

- pop density
- total land/water area
- nonwhite population
- total population
- share of metro thats urbanized?

also right now the network density measures use crude total area of the metro. Probably want to limit to land area or developed area

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
import os
import numpy as np

In [ ]:
files = [fn for fn in os.listdir("../data/metrics/") if fn.endswith("measures.csv")]

In [ ]:
dfs = []

for file in files:

    fips = file.split('_')[0]

    df = pd.read_csv(f"../data/metrics/{file}")
    df['msa'] = fips
    df['node_props_dead_end'] = df.streets_per_node_proportions.iloc[1]
    df['node_props_3way'] = df.streets_per_node_proportions.iloc[3]
    df['node_props_4way'] = df.streets_per_node_proportions.iloc[4]
    
    # I think these "node proportion" measures are equivalent to "degree" in network science. I think they're gonna be critical but not 10)% on
    # how to operationalize them... do we need each? aggregate >4?, etc.
    
    df = df.groupby('msa').first()
    df = df.drop(columns=['streets_per_node_proportions', 'streets_per_node_counts', 'Unnamed: 0'])
    dfs.append(df)
    df = pd.concat(dfs).reset_index()

In [ ]:
df = pd.concat(dfs)

In [ ]:
#df.seg_difference.hist()

In [ ]:
seg_diffs = pd.read_csv("../data/network_comparison.csv")

In [ ]:
seg_diffs.fips = seg_diffs.fips.astype(str)

In [ ]:
df = df.merge(seg_diffs, left_index=True, right_on='fips', how='inner')

In [ ]:
df.shape

In [ ]:
df.node_props_dead_end.hist()

In [ ]:
df.circuity_avg.apply(np.log).hist()

In [ ]:
import seaborn as sns

In [ ]:
df['pct_diff'] = (df.seg_difference / df.planar_measure) * 100

In [ ]:
df.loc[df.p_value <= 0.050, "significant"] = 1
df.significant=df.significant.fillna(0)

In [ ]:
df.pct_diff.hist()

In [ ]:
df = df.drop(columns=['Unnamed: 0']).set_index('name')

In [ ]:
df

In [ ]:
df_sub = df[df.p_value<=0.05]

In [ ]:
df.pct_diff.hist()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.columns

In [ ]:
features = ['n', 'm', 'k_avg', 'edge_length_total', 'edge_length_avg',
       'streets_per_node_avg', 'intersection_count', 'street_length_total',
       'street_segment_count', 'street_length_avg', 'circuity_avg',
       'self_loop_proportion', 'node_density_km', 'intersection_density_km',
       'edge_density_km', 'street_density_km', 'node_props_dead_end',
       'node_props_3way', 'node_props_4way']

In [ ]:
form = ("seg_difference ~ 1 + np.log(n) + np.log(m) +  + np.log(edge_length_avg) + streets_per_node_avg +" 
         "np.log(street_segment_count) + np.log(circuity_avg) + self_loop_proportion +"
        "np.log(intersection_density_km) + np.log(edge_density_km) + np.log(street_density_km) + node_props_dead_end ")

model = smf.ols(form, df).fit()

In [ ]:
model_subset = smf.ols(form, df_sub).fit()

In [ ]:
smf.glm?

In [ ]:
import statsmodels.api as sm

In [ ]:
model_log = smf.glm("significant ~ 1 + np.log(n) + np.log(m) +  + np.log(edge_length_avg) + streets_per_node_avg +" 
         "np.log(street_segment_count) + np.log(circuity_avg) + self_loop_proportion +"
        "np.log(intersection_density_km) + np.log(edge_density_km) + np.log(street_density_km) + node_props_dead_end ", df, family=sm.families.Binomial()).fit()

In [ ]:
model.summary()

In [ ]:
print(model_subset.summary())

In [ ]:
model_log.summary()

In [ ]:
model_log.resid_pearson

In [ ]:
sns.set_context('notebook')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
diagnostic = pd.DataFrame({'predicted':model_log.mu, 'actual':df.pct_diff, 'resid':model_log.resid_pearson})

In [ ]:
diagnostic.plot(kind='scatter', x='predicted', y='actual')

In [ ]:
df.iloc[2]

In [ ]:
from seg_networks import Linear_Reg_Diagnostic

In [ ]:
diag_plots = Linear_Reg_Diagnostic(model)

In [ ]:
diag_plots()

In [ ]:
diag_plots.vif_table().head()

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
cmat = df.corr()

In [ ]:
cmat.pct_diff


In [ ]:
cmat.planar_measure

In [ ]:
cmat.network_measure

In [ ]:
import seaborn as sns

In [ ]:
sns.displot(df, x='network_measure', y='seg_difference')

In [ ]:
sns.displot(df, x='network_measure', y='pct_diff')

In [ ]:
g = sns.lmplot(x='network_measure', y='pct_diff', data=df)

In [ ]:
g = sns.lmplot(x='planar_measure', y='network_measure', data=df)

In [ ]:
g = sns.lmplot(x='planar_measure', y='network_measure', data=df)
x0, x1 = g.ax.get_xlim()
y0, y1 = g.ax.get_ylim()
lims = [max(x0, y0), min(x1, y1)]
g.ax.plot(lims, lims, '--r');
plt.savefig("../paper/figures/scatter.png", dpi=600)

In [ ]:
g = sns.lmplot(x='n', y='pct_diff', data=df)

In [ ]:
g = sns.lmplot(x='street_length_total', y='pct_diff', data=df)

In [ ]:
g = sns.lmplot(x='n', y='street_length_total', data=df)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
results = smf.ols("pct_diff ~ n", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("pct_diff ~ n + planar_measure", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("pct_diff ~ n + planar_measure + significant", data=df).fit()

print(results.summary())

In [ ]:
g = sns.lmplot(data=df, x='planar_measure', y='pct_diff', hue='significant')

In [ ]:
g = sns.lmplot(data=df, x='n', y='pct_diff', hue='significant')

In [ ]:
g = sns.lmplot(data=df, x='circuity_avg', y='pct_diff', hue='significant')

In [ ]:
results = smf.ols("pct_diff ~ n + planar_measure + significant + circuity_avg", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("pct_diff ~ n + planar_measure + significant + circuity_avg", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("pct_diff ~ n + planar_measure + circuity_avg", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("planar_measure ~ n + circuity_avg", data=df).fit()

print(results.summary())

In [ ]:
results = smf.ols("network_measure ~ n + circuity_avg", data=df).fit()

print(results.summary())

In [ ]:
g = sns.lmplot(data=df, x='planar_measure', y='network_measure', hue='significant')

In [ ]:
df['ps_inter'] = df.significant * df.planar_measure

In [ ]:
results = smf.ols("network_measure ~ planar_measure + ps_inter", data=df).fit()

print(results.summary())

In [ ]:
(df.network_measure > df.planar_measure).sum()

In [ ]:
spvars = ['network_measure', 'planar_measure', 'significant', 'pct_diff', 'circuity_avg', 'street_length_total']

In [ ]:
sns.pairplot(df[spvars], hue='significant');

In [ ]:
def pcbar(df, name, output=None):
    cdf = df.corr()
    var_names = cdf.columns.values
    pairs = [ ]
    k = len(var_names)
    for i in range(k-1):
        for j in range(i+1, k):
            pair =  f'{var_names[i]} v. {var_names[j]}'
            c = cdf.iloc[i,j]
            #print(pair, c)
            pairs.append([pair, c])
    df = pd.DataFrame(pairs, columns=['pair', 'rho'])
    rho = df.loc[:, ['rho']]
    df['colors'] = ['red' if rho < 0 else 'green' for rho in df['rho']]
    df.sort_values('rho', inplace=True)
    df.reset_index(inplace=True)
    rho = df.loc[:, ['rho']]

    plt.figure(figsize=(14, 10), dpi=600)
    xmin = rho * (rho < 0) 
    xmax = rho * (rho > 0)
    
    
    plt.hlines(y = df.index, xmin=xmin, xmax=xmax, 
               colors=df.colors,
               alpha=0.4, linewidth=5)
    fontsize=12
    if k > 10:
        fontsize = 3
        
    plt.gca().set(ylabel='$pair$', xlabel='$rho$')
    plt.yticks(df.index, df.pair, fontsize=fontsize)
    plt.title(f'Pairwise Correlations {name}', fontdict={'size':20})
    plt.xlim(-1, 1)
    plt.grid(linestyle='--', alpha=0.5)
    if output:
        plt.savefig(output, dpi=600, figsize=(16,9), bbox_inches = 'tight')
    plt.show()
    return df

In [ ]:
pcbar(df, 'correlations')

In [ ]:
def pcbar(df, name, output=None, focal=None):
    cdf = df.corr()
    var_names = cdf.columns.values
    pairs = [ ]
    k = len(var_names)
    for i in range(k-1):
        for j in range(i+1, k):
            pair =  f'{var_names[i]} v. {var_names[j]}'
            c = cdf.iloc[i,j]
            #print(pair, c)
            if focal:
                if var_names[i]==focal or var_names[j]==focal:
                    pairs.append([pair, c])
            else:
                pairs.append([pair, c])
    df = pd.DataFrame(pairs, columns=['pair', 'rho'])
    rho = df.loc[:, ['rho']]
    df['colors'] = ['red' if rho < 0 else 'green' for rho in df['rho']]
    df.sort_values('rho', inplace=True)
    df.reset_index(inplace=True)
    rho = df.loc[:, ['rho']]

    plt.figure(figsize=(14, 10), dpi=600)
    xmin = rho * (rho < 0) 
    xmax = rho * (rho > 0)
    
    
    plt.hlines(y = df.index, xmin=xmin, xmax=xmax, 
               colors=df.colors,
               alpha=0.4, linewidth=5)
    fontsize=12
    if len(pairs) > 50:
        fontsize = 3
        
    plt.gca().set(ylabel='$pair$', xlabel='$rho$')
    plt.yticks(df.index, df.pair, fontsize=fontsize)
    plt.title(f'Pairwise Correlations {name}', fontdict={'size':20})
    plt.xlim(-1, 1)
    plt.grid(linestyle='--', alpha=0.5)
    if output:
        plt.savefig(output, dpi=600, figsize=(16,9), bbox_inches = 'tight')
    plt.show()
    return df

In [ ]:
pcbar(df, 'pct_diff', focal='pct_diff', output="../paper/figures/correlations.png")

In [ ]:
pcbar(df, 'seg_differrence', focal='seg_difference')

In [ ]:
# plot all the metros that have significant differences
from geosnap import DataStore
DataStore().msas().merge(df, left_on='geoid', right_on='fips').explore('significant', categorical=True, cmap='plasma')

just about all the large metros are significant

In [ ]:
DataStore().msas()[DataStore().msas()['type']=='Metro Area'].plot()

In [ ]:
DataStore().msas()

In [ ]:
import sklearn

In [ ]:
pca = sklearn.decomposition.PCA().fit(df[features])

In [ ]:
from seg_networks import best_factors

In [ ]:
df[features].shape[1]

In [ ]:
best_factors(df[features])

In [ ]:
pca = sklearn.decomposition.PCA(n_components=6).fit(df[features])

In [ ]:
pca.transform(df[features])

In [ ]:
import statsmodels.api as sm

In [ ]:
sm.OLS(df.seg_difference, pca.transform(df[features])).fit().summary()

In [ ]:
Linear_Reg_Diagnostic(sm.OLS(df.pct_diff, pca.transform(df[features])).fit())()